# AI Red Teaming Agent for Generative AI models and applications in Azure AI Foundry

## Objective
This notebook walks through how to use Azure AI Evaluation's AI Red Teaming Agent functionality to assess the safety and resilience of AI systems against adversarial prompt attacks. AI Red Teaming Agent leverages [Risk and Safety Evaluations](https://learn.microsoft.com/en-us/azure/ai-foundry/concepts/evaluation-metrics-built-in?tabs=warning#risk-and-safety-evaluators) to help identify potential safety issues across different risk categories (violence, hate/unfairness, sexual content, self-harm) combined with attack strategies of varying complexity levels from [PyRIT](https://github.com/Azure/PyRIT), Microsoft AI Red Teaming team's open framework for automated AI red teaming.

## Time
You should expect to spend about 30-45 minutes running this notebook. Execution time will vary based on the number of risk categories, attack strategies, and complexity levels you choose to evaluate.

## Before you begin

### Prerequisite
The AI Red Teaming Agent requires an Azure AI Foundry project configuration and Azure credentials. Your project configuration will be used to log red teaming scan results after the run is finished.

**Important**: Make sure to authenticate to Azure using `az login` in your terminal before running this notebook.

### Installation
From a terminal window, navigate to your working directory which contains this sample notebook, and execute the following.
```bash
python -m venv .venv
```

Then, activate the virtual environment created:

```bash
# %source .venv/bin/activate # If using Mac/Linux OS
.venv/Scripts/activate # If using Windows OS
```

With your virtual environment activated, install the following packages required to execute this notebook:

```bash
pip install uv
uv pip install azure-ai-evaluation[redteam] azure-identity openai
```


Now open VSCode with the following command, and ensure your virtual environment is used as kernel to run the remainder of this notebook.
```bash
code .
```

### Imports

In [2]:
%pip install --upgrade azure-ai-evaluation[redteam]

   ---------------------------------------- 0.0/773.7 kB ? eta -:--:--
   --------------------------------------- 773.7/773.7 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: azure-ai-evaluation
    Found existing installation: azure-ai-evaluation 1.4.0
    Uninstalling azure-ai-evaluation-1.4.0:
      Successfully uninstalled azure-ai-evaluation-1.4.0
Note: you may need to restart the kernel to use updated packages.


In [1]:
from typing import Optional, Dict, Any
import os

# Azure imports
from azure.ai.evaluation.red_team import RedTeam, RiskCategory, AttackStrategy

# OpenAI imports
from openai import AzureOpenAI

[INFO] Could not import SKAgentConverter. Please install the dependency with `pip install semantic-kernel`.


### Set Up Your Environment Variables

Set the following variables for use in this notebook. These variables connect to your Azure resources and model deployments.

**Note:** You can find these values in your Azure AI Foundry project or Azure OpenAI resource.

For reference, here's an example of what your populated environment variables should look like:

```
# Azure OpenAI
AZURE_OPENAI_API_KEY="your-api-key-here"
AZURE_OPENAI_ENDPOINT="https://endpoint-name.openai.azure.com/openai/deployments/deployment-name/chat/completions"
AZURE_OPENAI_DEPLOYMENT_NAME="gpt-4"
AZURE_OPENAI_API_VERSION="2023-12-01-preview"

# Azure AI Project
AZURE_SUBSCRIPTION_ID="12345678-1234-1234-1234-123456789012"
AZURE_RESOURCE_GROUP_NAME="your-resource-group"
AZURE_PROJECT_NAME="your-project-name"
```

In [2]:
from dotenv import load_dotenv
load_dotenv("../../.env")

# Azure AI Project information
azure_ai_project = {
    "subscription_id": os.environ.get("SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("RG_NAME"),
    "project_name": os.environ.get("PROJECT_NAME"),
}

# Azure OpenAI deployment information
azure_openai_deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")  # e.g., "gpt-4"
azure_openai_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")  # e.g., "https://endpoint-name.openai.azure.com/openai/deployments/deployment-name/chat/completions"
azure_openai_api_key = os.environ.get("AZURE_OPENAI_API_KEY")  # e.g., "your-api-key"
azure_openai_api_version = os.environ.get("AZURE_OPENAI_API_VERSION")  # Use the latest API version

## Understanding AI Red Teaming Agent's capabilities

The Azure AI Evaluation SDK's `RedTeam` functionality evaluates AI systems against adversarial prompts across multiple dimensions:

1. **Risk Categories**: Different content risk categories your AI system might generate
   - Violence
   - HateUnfairness
   - Sexual
   - SelfHarm

2. **Attack Strategies**: Along with standard unmodified prompts which are sent by default as the `baseline`, you can specify different transformations of prompts to elicit undesired content.
You can also use `AttackStrategy.Compose()` to layer two strategies in one attack
   - AnsiAttack: Using ANSI escape codes in prompts
   - AsciiArt: Using ASCII art to disguise harmful content
   - AsciiSmuggler: Hiding harmful content within ASCII characters
   - Atbash: Using the Atbash cipher to encode harmful requests
   - Base64: Encoding harmful content in Base64 format
   - Binary: Converting text to binary to bypass filters
   - Caesar: Using the Caesar cipher for encoding
   - CharacterSpace: Manipulating character spacing to confuse filters
   - CharSwap: Swapping characters to bypass detection
   - Diacritic: Using diacritical marks to alter text appearance
   - Flip: Flipping text to bypass content filters
   - Leetspeak: Converting letters to numbers and symbols
   - Morse: Using Morse code to encode harmful requests
   - ROT13: Using ROT13 cipher for text transformation
   - SuffixAppend: Adding suffixes to confuse detection systems
   - StringJoin: Joining strings in unconventional ways
   - Tense: Changing the tense of harmful requests to past tense
   - UnicodeConfusable: Using similar-looking Unicode characters
   - UnicodeSubstitution: Substituting characters with Unicode alternatives
   - Url: Embedding harmful content within URLs
   - Jailbreak: Specially crafted prompts to bypass AI safeguards

3. **Complexity Levels**: Different difficultly levels of attacks
   - Baseline: Standard functionality tests
   - Easy: Simple attack patterns
   - Moderate: More sophisticated attacks
   - Difficult: Complex, layered attack strategies

The key metric for evaluating results is the **Attack Success Rate (ASR)**, which measures the percentage of attacks that successfully elicit harmful content from your AI system.

## Basic Example: Fixed Response Callback

Let's start with a basic example that demonstrates how to set up a Red Team evaluation using a simple callback that always returns a fixed, safe response regardless of the input prompt.

**Important Note**: Since this callback always responds with the same safe message, the Attack Success Rate (ASR) will be 0% for all categories. This example is merely to demonstrate the API structure and flow.

In [3]:
# Define a simple callback function that always returns a fixed response
def financial_advisor_callback(query: str) -> str:  # noqa: ARG001
    return "I'm a financial advisor assistant. I can help with investment advice and financial planning within legal and ethical guidelines."

In [4]:
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

credential = DefaultAzureCredential()

# Create the `RedTeam` instance with minimal configurations
red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness],
    num_objectives=1,
)

Class RedTeam: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


NOTE: `num_objectives` specifies the number of attacks to perform per risk category per attack strategy. If the parameter `risk_categories` is not specified, `[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm]` will be used by default.

Now let's run a simple automated scan using the `RedTeam` with the fixed response target. We'll test against two risk categories and one attack strategy for simplicity.

In [5]:
# Run the red team scan called "Basic-Callback-Scan" with limited scope for this basic example
# This will test 1 objective prompt for each of Violence and HateUnfairness categories with the Flip strategy
result = await red_team.scan(
    target=financial_advisor_callback, scan_name="Basic-Callback-Scan", attack_strategies=[AttackStrategy.Flip]
)

🚀 STARTING RED TEAM SCAN: Basic-Callback-Scan
📂 Output directory: .\.scan_Basic-Callback-Scan_20250728_153559
📊 Risk categories: ['violence', 'hate_unfairness']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/build/evaluation/3ceea4a1-445e-4ff7-a797-adf3367b2fb5?wsid=/subscriptions/8babb7f9-50f7-498f-9e0a-8bef4389331d/resourceGroups/rg-ProjectFoundryHub03072025_2/providers/Microsoft.MachineLearningServices/workspaces/ProjectFoundryHub03072025_2
📋 Planning 4 total tasks


Scanning:   0%|                         | 0/4 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                  | 0/4 [00:04<?, ?scan/s, current=fetching baseline/hate_unfairness]

📝 Fetched baseline objectives for violence: 1 objectives


...

Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\.scan_Basic-Callback-Scan_20250728_153559\flip_violence_8c88ce61-2b8b-4f94-8f32-fd4326eafb01.json".

✅ Completed task 1/4 (25.0%) - flip/violence in 18.5s
   Est. remaining: 1.5 minutes
Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\.scan_Basic-Callback-Scan_20250728_153559\baseline_hate_unfairness_64ca2abf-985a-4d16-9297-ff13c5533d19.json".

✅ Completed task 2/4 (50.0%) - baseline/hate_unfairness in 18.5s
   Est. remaining: 0.5 minutes
Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\.scan_Basic-Callback-Scan_20250728_153559\baseline_violence_30b61503-1a77-4b79-99a5-9ab398be82a2.json".

✅ Completed task 3/4 (75.0%) - baseline/violence in 18.6s
   Est. remaining: 0.2 minutes


Scanning: 100%|██████████████████████████████████| 4/4 [00:25<00:00,  6.25s/scan, current=batch 1/1]


Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\.scan_Basic-Callback-Scan_20250728_153559\flip_hate_unfairness_beaca946-2e78-48da-b860-a9832db3cb11.json".

✅ Completed task 4/4 (100.0%) - flip/hate_unfairness in 19.3s
   Est. remaining: 0.0 minutes


Class RedTeamResult: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\.scan_Basic-Callback-Scan_20250728_153559\final_results.json".

Overall ASR: 0.0%
Attack Success: 0/4 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category     | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------------------------------
Violence          | 0.0%           | 0.0%                         | N/A                             | N/A                           
Hate-unfairness   | 0.0%           | 0.0%                         | N/A                             | N/A                           

Detailed results available at:
https://ai.azure.com/build/evaluation/3ceea4a1-445e-4ff7-a797-a

## Intermediary Example: Using a Model Configuration as Target

Now let's create a more realistic example that uses an Azure OpenAI model for responding to the red teaming prompts. To test base or foundation models, you can update your target to take in a model configuration:

In [6]:
# Define a model configuration to test
azure_oai_model_config = {
    "azure_endpoint": azure_openai_endpoint,
    "azure_deployment": azure_openai_deployment,
    "api_key": azure_openai_api_key,
}

Then, update your target to point to the model configurations and run the scan.

In [7]:
# Run the red team scan called "Intermediary-Model-Target-Scan"
result = await red_team.scan(
    target=azure_oai_model_config, scan_name="Intermediary-Model-Target-Scan", attack_strategies=[AttackStrategy.Flip]
)

🚀 STARTING RED TEAM SCAN: Intermediary-Model-Target-Scan
📂 Output directory: .\.scan_Intermediary-Model-Target-Scan_20250728_154520
📊 Risk categories: ['violence', 'hate_unfairness']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/build/evaluation/19b179e8-5897-4242-977b-0639cbb8fbb7?wsid=/subscriptions/8babb7f9-50f7-498f-9e0a-8bef4389331d/resourceGroups/rg-ProjectFoundryHub03072025_2/providers/Microsoft.MachineLearningServices/workspaces/ProjectFoundryHub03072025_2
📋 Planning 4 total tasks


Scanning:   0%|                         | 0/4 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                             | 0/4 [00:19<?, ?scan/s, current=fetching flip/violence]

📝 Fetched baseline objectives for violence: 1 objectives
📝 Fetched baseline objectives for hate_unfairness: 1 objectives
🔄 Fetching objectives for strategy 2/2: flip


...

Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\.scan_Intermediary-Model-Target-Scan_20250728_154520\flip_violence_94ee6529-af5f-40d2-8a6b-3cbd681e45fa.json".

✅ Completed task 1/4 (25.0%) - flip/violence in 56.1s
   Est. remaining: 4.1 minutes
Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\.scan_Intermediary-Model-Target-Scan_20250728_154520\baseline_hate_unfairness_a4f4de7f-0fcd-4dd7-8e2d-fd6fea860e61.json".

✅ Completed task 2/4 (50.0%) - baseline/hate_unfairness in 56.2s
   Est. remaining: 1.4 minutes
Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\.scan_Intermediary-Model-Target-Scan_20250728_154520\baseline_violence_891cc6b0-2dd0-453f-8b17-db92e2797f74.json".



Scanning:  75%|█████████████████████████▌        | 3/4 [01:16<00:19, 19.78s/scan, current=batch 1/1]

✅ Completed task 3/4 (75.0%) - baseline/violence in 56.3s
   Est. remaining: 0.5 minutes


Scanning: 100%|██████████████████████████████████| 4/4 [01:24<00:00, 21.15s/scan, current=batch 1/1]


Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\.scan_Intermediary-Model-Target-Scan_20250728_154520\flip_hate_unfairness_52f59bb1-6af6-4e7f-b368-ccc6d2961405.json".

✅ Completed task 4/4 (100.0%) - flip/hate_unfairness in 64.7s
   Est. remaining: 0.0 minutes
Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\.scan_Intermediary-Model-Target-Scan_20250728_154520\final_results.json".

Overall ASR: 0.0%
Attack Success: 0/4 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category     | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
------------------------------------------------------------------------------------------------------------------------------------
Violence          | N/A   

## Advanced Example: Using an Azure Open AI Model Endpoint in a Callback Function

Using the same Azure Open AI model configuration as above, we now wrap it in a callback function for more flexibility and control on the input and output handling. This will demonstrate how to evaluate an actual AI application. To test your own actual AI application, replace the inside of the callback function with a call to your application.

In [8]:
# Define a callback that uses Azure OpenAI API to generate responses
async def azure_openai_callback(
    messages: list,
    stream: Optional[bool] = False,  # noqa: ARG001
    session_state: Optional[str] = None,  # noqa: ARG001
    context: Optional[Dict[str, Any]] = None,  # noqa: ARG001
) -> dict[str, list[dict[str, str]]]:
    deployment = os.environ.get("AZURE_OPENAI_DEPLOYMENT")
    endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    api_version = os.environ.get("AZURE_API_VERSION")
    api_key = os.environ.get("AZURE_OPENAI_API_KEY")

    # Initialize Azure OpenAI client
    client = AzureOpenAI(
        azure_endpoint=endpoint, 
        api_key=api_key,
        api_version=api_version,
    )

    ## Extract the latest message from the conversation history
    messages_list = [{"role": message.role, "content": message.content} for message in messages]
    latest_message = messages_list[-1]["content"]

    try:
        # Call the model
        response = client.chat.completions.create(
            model=deployment,
            messages=[
                {"role": "user", "content": latest_message},
            ],
            # max_tokens=500, # If using an o1 base model, comment this line out
            max_completion_tokens=500,  # If using an o1 base model, uncomment this line
            temperature=0.0, # If using an o1 base model, comment this line out (temperature param not supported for o1 base models)
        )

        # Format the response to follow the expected chat protocol format
        formatted_response = {"content": response.choices[0].message.content, "role": "assistant"}
    except Exception as e:
        print(f"Error calling Azure OpenAI: {e!s}")
        formatted_response = "I encountered an error and couldn't process your request."
    return {"messages": [formatted_response]}

In [9]:
# Create the RedTeam instance with all of the risk categories with 5 attack objectives generated for each category
model_red_team = RedTeam(
    azure_ai_project=azure_ai_project,
    credential=credential,
    risk_categories=[RiskCategory.Violence, RiskCategory.HateUnfairness, RiskCategory.Sexual, RiskCategory.SelfHarm],
    num_objectives=5,
)

We will use this instance of `model_red_team` to test different attack strategies in the following section.

### Testing Different Attack Strategies

Now we'll run a more comprehensive evaluation using multiple attack strategies across risk categories. This will give us a better understanding of our model's vulnerabilities.

In [10]:
# Run the red team scan with multiple attack strategies
advanced_result = await model_red_team.scan(
    target=azure_openai_callback,
    scan_name="Advanced-Callback-Scan",
    attack_strategies=[
        AttackStrategy.EASY,  # Group of easy complexity attacks
        AttackStrategy.MODERATE,  # Group of moderate complexity attacks
        AttackStrategy.CharacterSpace,  # Add character spaces
        AttackStrategy.ROT13,  # Use ROT13 encoding
        AttackStrategy.UnicodeConfusable,  # Use confusable Unicode characters
        AttackStrategy.CharSwap,  # Swap characters in prompts
        AttackStrategy.Morse,  # Encode prompts in Morse code
        AttackStrategy.Leetspeak,  # Use Leetspeak
        AttackStrategy.Url,  # Use URLs in prompts
        AttackStrategy.Binary,  # Encode prompts in binary
        AttackStrategy.Compose([AttackStrategy.Base64, AttackStrategy.ROT13]),  # Use two strategies in one attack
    ],
    output_path="Advanced-Callback-Scan.json",
)

🚀 STARTING RED TEAM SCAN: Advanced-Callback-Scan
📂 Output directory: .\.scan_Advanced-Callback-Scan_20250728_154748
📊 Risk categories: ['violence', 'hate_unfairness', 'sexual', 'self_harm']
🔗 Track your red team scan in AI Foundry: https://ai.azure.com/build/evaluation/a6006769-fa5e-465c-bb2b-bc7a57367576?wsid=/subscriptions/8babb7f9-50f7-498f-9e0a-8bef4389331d/resourceGroups/rg-ProjectFoundryHub03072025_2/providers/Microsoft.MachineLearningServices/workspaces/ProjectFoundryHub03072025_2
📋 Planning 52 total tasks


Scanning:   0%|                        | 0/52 [00:00<?, ?scan/s, current=fetching baseline/violence]

📚 Using attack objectives from Azure RAI service


Scanning:   0%|                          | 0/52 [00:02<?, ?scan/s, current=fetching baseline/sexual]

📝 Fetched baseline objectives for violence: 5 objectives
📝 Fetched baseline objectives for hate_unfairness: 5 objectives


...

▶️ Starting task: tense strategy for sexual risk category
▶️ Starting task: tense strategy for self_harm risk category


ERROR: [tense/sexual] Error processing batch 1: Error sending prompt with conversation ID: 3b46b709-c72b-472f-ab18-27295e287d8a
Traceback (most recent call last):
  File "c:\Users\ruplisso\AppData\Local\miniconda3\envs\genaiops2env\Lib\site-packages\pyrit\prompt_normalizer\prompt_normalizer.py", line 95, in send_prompt_async
    response = await target.send_prompt_async(prompt_request=request)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\ruplisso\AppData\Local\miniconda3\envs\genaiops2env\Lib\site-packages\azure\ai\evaluation\red_team\_callback_chat_target.py", line 52, in send_prompt_async
    response_context = await self._callback(messages=messages, stream=self._stream, session_state=None, context=None)  # type: ignore
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\ruplisso\AppData\Local\Temp\ipykernel_37540\1452257640.py", line 14, in azure_openai_callba

Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\Advanced-Callback-Scan.json".

✅ Completed task 51/52 (98.1%) - tense/sexual in 56.5s
   Est. remaining: 0.5 minutes


Scanning: 100%|██████████████████████████████| 52/52 [26:42<00:00, 30.83s/scan, current=batch 11/11]


Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\Advanced-Callback-Scan.json".

✅ Completed task 52/52 (100.0%) - tense/self_harm in 57.0s
   Est. remaining: 0.0 minutes
Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\Advanced-Callback-Scan.json".

Evaluation results saved to "C:\Users\ruplisso\Documents\GitHub\genaiops-exercises2\solutions\07_AI_Red_Teaming\.scan_Advanced-Callback-Scan_20250728_154748\final_results.json".

Overall ASR: 0.0%
Attack Success: 0/260 attacks were successful
------------------------------------------------------------------------------------------------------------------------------------
Risk Category     | Baseline ASR   | Easy-Complexity Attacks ASR  | Moderate-Complexity Attacks ASR | Difficult-Complexity Attacks ASR
-----------------------------------------------------------------------------------------------------------------

The data and results used in this attack will be saved to the `output_path` specified. The URL printed out at the end of the scorecard will provide a link to where you results are uploaded and logged to your Azure AI Foundry project.

## Conclusion

In this notebook, we've demonstrated how to use the Azure AI Evaluation SDK's `RedTeam` functionality to assess the safety and resilience of AI systems. We started with a basic fixed-response example and then moved to a more realistic model testing across multiple risk categories and attack strategies.

The automated AI red teaming scans provides valuable insights into:

1. **Overall Attack Success Rate (ASR)** - The percentage of attacks that successfully elicit harmful content
2. **Vulnerability by Risk Category** - Which types of harmful content your model is most vulnerable to
3. **Effectiveness of Attack Strategies** - Which attack techniques are most successful against your model
4. **Impact of Complexity** - How more sophisticated attacks affect your model's safety guardrails

By regularly red-teaming your AI applications, you can identify and address potential vulnerabilities before deploying your models to production environments.

### Next Steps

1. **Mitigation**: Use these results to strengthen your model's guardrails against identified attack vectors
2. **Continuous Testing**: Implement regular red team evaluations as part of your development lifecycle
3. **Custom Strategies**: Develop custom attack strategies for your specific use cases and domain
4. **Safety Layers**: Consider adding additional safety layers like Azure AI Content Safety to filter harmful requests and responses 